In [134]:
import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [82]:
domicile = []
exterieur = []
score = []
date = []
saison = []
game_tag = []

def convert_date(date_string):
    date_obj = datetime.strptime(date_string, '%b %d, %Y')
    formatted_date = date_obj.strftime('%d/%m/%Y')
    return formatted_date
    

for annee in range(2000, 2024):

    dom = ext = sco = last_day = day = x = 0
    
    url = "https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/"+str(annee)
    print(url)

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }

    response = requests.get(url, headers = headers)
    #response.status_code
    soup = BeautifulSoup(response.content, "html.parser")

    date_erronee = soup.select('td[class="hide-for-small"]')

    dom = soup.select('td[class="text-right no-border-rechts hauptlink"]')
    ext = soup.select('td[class="no-border-links hauptlink"]')
    sco = soup.select('td[class="zentriert hauptlink"]')

    for i in range(len(dom)):
        
        domicile.append(dom[i].text.split('\xa0')[-1].strip())
        exterieur.append(ext[i].text.split('\xa0')[0].strip())
        score.append(sco[i].text.split('\xa0')[1].strip())
        saison.append(annee)
        
        a_tag = sco[i].find('a')
        id_value = a_tag.get('id')
        game_tag.append(id_value)
        
        day = date_erronee[i].text
        
        if day.strip() == "" :
            date.append(last_day)
        else:
            if annee < 2000:
                x = (re.search(r'%b %d, %Y', day))
                date.append(convert_date(x))
                last_day = date[i]
            if annee >= 2000:
                date.append(re.search(r'\d+/\d+/\d+', day).group())
                last_day = date[i]

##for i in range(len(domicile)):
#    print(date[i],":",domicile[i],score[i],exterieur[i])

https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2000
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2001
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2002
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2003
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2004
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2005
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2006
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2007
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2008
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbewerb/GB1/saison_id/2009
https://www.transfermarkt.co.uk/premier-league/gesamtspielplan/wettbew

In [87]:
import csv
import os

nom_fichier = 'games.csv'

if os.path.exists(nom_fichier):
    os.remove(nom_fichier)

entetes = ['game_tag', 'season', 'date', 'domicile_team', 'score', 'exterieur_team']
data = []
for i in range(len(domicile)):
    x = {
        'game_tag': game_tag[i],
        'season': saison[i],
        'date': date[i],
        'domicile_team': domicile[i],
        'score': score[i],
        'exterieur_team': exterieur[i]
    }
    data.append(x)

with open(nom_fichier, mode='w', newline='', encoding='utf-8') as fichier_csv:
    writer = csv.DictWriter(fichier_csv, fieldnames=entetes)
    writer.writeheader()
    for element in data:
        writer.writerow(element)


Le fichier a été supprimé avec succès.
